In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import sparse
from scipy.stats import pearsonr
import os
import csv

In [13]:
# Parameters
N = 200                                   
N2 = N // 2
p = 0.3
dt = 0.1
itmax = 2000
nloop = 16
sigman = 1
b = 0.5
gsyn = 0.5
nloop_train = 10
itstim = 200
amp_corriente = 6
ftrain = 1
alpha = 0.25
amp0 = 4
vt = 0.5

cant_seed = 2


iout = np.linspace(0, N, num=N, endpoint=False).astype(int)
i_graph = np.array([0, 1, 2, N2 + 1, N2 + 2, N2 + 3])

In [14]:
def crear_archivo_parametros(filename_resultados, num_simulacion, nombre_carpeta):
 #file donde guardo los parámetros de la simulación
    data_parametros = {
        'N': [N],
        'p': [p],
        'gsyn': [gsyn],
        'nloop': [nloop],
        'nloop_train':[nloop_train],
        'cant_seed': [cant_seed],
        'dt': [dt],
        'itmax': [itmax],
        'itstim': [itstim],
        'amp_corriente': [amp_corriente],
        'ftrain': [ftrain],
        'alpha': [alpha],
        'sigman': [sigman],
        'vt': [vt],
        'b': [b],
        'results_file': [filename_resultados],
    }


    df = pd.DataFrame(data_parametros)
    filename_parametros = f'simulacion_{num_simulacion}_parametros.csv'
    csv_parametros_path = os.path.join(nombre_carpeta, filename_parametros)
    df.to_csv(csv_parametros_path, index=False)



In [15]:
def dynamics(x_var, r_var, I_var, nqif):
    dx = np.zeros(N)
    # LIF
    dx[nqif:] = -x_var[nqif:] + I_var[nqif:] + np.random.randn(N - nqif)*sigman
    # QIF
    dx[:nqif] = 1 - np.cos(x_var[:nqif]) + I_var[:nqif] * (1 + np.cos(x_var[:nqif])) + np.random.randn(nqif)*sigman

    dr = -b*r_var
    return dx, dr



def detect(x, xnew, rnew, nspike, nqif, vt):
    # LIF
    ispike_lif = np.where((x[nqif:] < vt) & (xnew[nqif:] > vt))[0] + nqif
    if(len(ispike_lif) > 0):
        rnew[ispike_lif[:]] +=  b
        xnew[ispike_lif[:]] = 0
        nspike[ispike_lif[:]] += 1
    # QIF
    dpi = np.mod(np.pi - np.mod(x, 2*np.pi), 2*np.pi)  # distancia a pi
    ispike_qif = np.where((xnew[:nqif] - x[:nqif] > 0) & (xnew[:nqif] - x[:nqif] - dpi[:nqif] > 0))[0]
    if(len(ispike_qif) > 0):
        rnew[ispike_qif[:]] += b
        nspike[ispike_qif[:]] += 1
    return xnew, rnew, nspike

In [16]:
def initialize_connectivity_matrix(N, p, gsyn):
    w = sparse.random(N, N, p, data_rvs=np.random.randn).todense()
    np.fill_diagonal(w, 0)  # No autapses
    w *= gsyn / np.sqrt(p * N)
    
    for i in range(N):
        i0 = np.where(w[i, :])[1]
        if len(i0) > 0:
            av0 = np.sum(w[i, i0]) / len(i0)
            w[i, i0] -= av0
    
    return w

def initialize_neurons(N):
    x = np.random.uniform(size=N) * 2 * np.pi
    r = np.zeros(N)
    nspike = np.zeros(N)

    return x, r, nspike

def initialize_training(N, w):
    # matrices de correlacion de las entradas
    nind=np.zeros(N).astype('int')
    idx=[]
    P=[]
    for i in range(N):
        ind=np.where(w[i,:])[1]
        nind[i]=len(ind)
        idx.append(ind)
        P.append(np.identity(nind[i])/alpha)   
    return P, idx

def currents(N, itmax):
    Iext=np.zeros((N,itmax))
    Ibac=amp_corriente*(2*np.random.uniform(size=N)-1)

    Iext[:, :itstim] = Ibac[:, None]  # Vectorized assignment

    return Iext


In [17]:
def generate_target():

    target=np.zeros((N,itmax))

    #amp=np.random.uniform(size=N)*amp0
    phase=np.random.uniform(size=N)*2*np.pi
    amp=np.random.uniform(size=N)*amp0
    for it in range(itmax):
        target[:,it] = amp*np.cos(2*np.pi/itmax*np.ones(N)*it+phase)
            
    return target 


In [18]:
def motifs(w, gsyn, N):
    w = w - np.mean(w)
    ww = w @ w
    wtw = w.T @ w
    wwt = w @ w.T
    sigma2 = np.trace(wwt) / N
    tau_rec = np.trace(ww) / (sigma2 * N)
    tau_div = (np.sum(wwt) - np.trace(wwt)) / (sigma2 * N * (N - 1))
    tau_con = (np.sum(wtw) - np.trace(wtw)) / (sigma2 * N * (N - 1))
    tau_chn = 2 * (np.sum(ww) - np.trace(ww)) / (sigma2 * N * (N - 1))
    return sigma2, tau_rec, tau_div, tau_con, tau_chn



In [19]:
def learning(it, w, r, P, idx, target):
    error = target[:, it:it + 1] - w @ r.reshape(N, 1)
    for i in range(N):
        ri = r[idx[i]].reshape(len(idx[i]), 1)
        k1 = P[i] @ ri
        k2 = ri.T @ P[i]
        den = 1 + ri.T @ k1
        P[i] -= (k1 @ k2) / den
        w[i, idx[i]] += (error[i, 0] * P[i] @ r[idx[i]])
    return w, P

In [20]:
# graficos
def plots(iplot, path, amp0, target, iout, pqif, rout):
    
    fig = plt.figure(figsize=(16,4))

    sub1 = fig.add_subplot(1,4,1)
    sub1.title.set_text('Target')
    sub1.set_xlim(itstim,itmax)
    sub1.set_ylim(-amp0,amp0)
    sub1.plot(target[iout[iplot],:])

    sub2 = fig.add_subplot(1,4,2)
    sub2.title.set_text(f'actividad pre-training, ipqif = {pqif}')
    sub2.set_xlim(itstim,itmax)
    sub2.plot(rout[:,iplot])

    sub3 = fig.add_subplot(1,4,3)
    sub3.title.set_text('actividad último loop de training')
    sub3.set_xlim((nloop_train)*itmax+itstim,(nloop_train+1)*itmax)

    sub3.plot(rout[:,iplot])
    sub4 = fig.add_subplot(1,4,4)
    sub4.title.set_text('actividad post-training')
    sub4.set_xlim((nloop-1)*itmax+itstim,nloop*itmax)
    sub4.plot(rout[:,iplot])


    plt.savefig(path+str(iplot)+'.png')
    plt.close(fig)

In [21]:
def crear_subcarpeta(nombre_carpeta, nombre_subcarpeta):
    subcarpeta_path_total = os.path.join(nombre_carpeta, nombre_subcarpeta)
    os.makedirs(subcarpeta_path_total, exist_ok=True)
    return subcarpeta_path_total

def crear_carpetas(num_simulacion):
    nombre_carpeta = f"simulacion_{num_simulacion}"
    os.makedirs(nombre_carpeta, exist_ok=True)
    nombre_subcarpeta_act = f"simulacion_{num_simulacion}_ejemplos_actividad"
    nombre_subcarpeta_pesos = f"simulacion_{num_simulacion}_matrices_pesos"
    nombre_subcarpeta_corrientes = f"simulacion_{num_simulacion}_corrientes"
    sub_act = crear_subcarpeta(nombre_carpeta, nombre_subcarpeta_act)
    sub_pesos = crear_subcarpeta(nombre_carpeta, nombre_subcarpeta_pesos)
    sub_corrientes = crear_subcarpeta(nombre_carpeta, nombre_subcarpeta_corrientes)
    return nombre_carpeta, sub_act, sub_pesos, sub_corrientes

def guardar_matriz_csv(matriz, nombre_archivo):
    np.savetxt(nombre_archivo, matriz, delimiter=',', fmt='%s')



In [22]:
num_simulacion = 22

nombre_carpeta, nombre_subcarpeta_act, nombre_subcarpeta_pesos, nombre_subcarpeta_corrientes = crear_carpetas(num_simulacion)
    
target = generate_target()
#file donde voy a guardar los resultados (CC, taus)
filename_resultados = f'simulacion_{num_simulacion}_resultados.csv'
csv_file_path = os.path.join(nombre_carpeta, filename_resultados)
column_names = [ 'pqif' ,'seed','nloop', 'cc_lif', 'cc_qif', 'cc', 'tau_rec','tau_div','tau_con','tau_chn']


crear_archivo_parametros(filename_resultados, num_simulacion, nombre_carpeta)
 

    
pqif_values = [0.5]


with open(csv_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if file.tell() == 0:
        writer.writerow(column_names)

    for pqif in pqif_values:
        
        path_corrientes_seed = os.path.join(nombre_subcarpeta_corrientes, f'simulacion_{num_simulacion}_corrientes_pqif_{pqif}.csv')

        with open(path_corrientes_seed, mode='a', newline='') as file_:
            writer_ = csv.writer(file_)
            if file_.tell() == 0:
                writer_.writerow(['pqif', 'seed', 'iloop', 'it', 'II_0', 'v_0', 'II_1', 'v_1', 'II_N2+1', 'v_N2+1', 'II_N2+2', 'v_N2+2'])

            nqif = int(N * pqif)

            for seed in range(cant_seed):
                np.random.seed(seed)
                x, r, nspike = initialize_neurons(N)
                rout = np.zeros((1, N))
                Iext = currents(N, itmax)
                w = initialize_connectivity_matrix(N, p, gsyn)
                P, idx = initialize_training(N, w)

                for iloop in range(nloop):
                    for it in range(itmax):
                        II = np.squeeze(np.asarray(Iext[:, it]))
                        v = w.dot(r.T).A1
                        dx, dr = dynamics(x, r, II + v, nqif)
                        xnew = x + dt * dx / 2
                        rnew = r + dt * dr / 2
                        dx, dr = dynamics(xnew, rnew, II + v, nqif)
                        xnew = x + dt * dx
                        rnew = r + dt * dr
                        xnew, rnew, nspike = detect(x, xnew, rnew, nspike, nqif, vt)
                        x, r = np.copy(xnew), np.copy(rnew)
                        rout = np.vstack([rout, r[iout]])

                        if iloop in [nloop_train + 1, nloop - 1] and it % 20 == 0:
                            writer_.writerow([pqif, seed, iloop, it, II[0], v[0], II[1], v[1], II[N2+1], v[N2+1], II[N2+2], v[N2+2]])

                    if iloop > 0 and iloop <= nloop_train and int(it > itstim):
                        w, P = learning(it, w, r, P, idx, target)

                    sigma2, tau_rec, tau_div, tau_con, tau_chn = motifs(w, gsyn, N)

                    if iloop in [0, nloop_train + 1]:
                        path_w_seed = os.path.join(nombre_subcarpeta_pesos, f'simulacion_{num_simulacion}_pesos_pqif_{pqif}_matriz_iloop_{iloop}_semilla_{seed}')
                        guardar_matriz_csv(w, path_w_seed)

                    valid_indices = np.where((np.var(target[:, itstim:], axis=1) > 0) & (np.var(rout[1+itstim+iloop*itmax:1+itstim+(iloop+1)*itmax, :], axis=0) > 0))[0]
                    ci = 0
                    for i in valid_indices:
                        m1 = 1 + itstim + iloop * itmax
                        m2 = m1 + itmax - itstim
                        ci += pearsonr(target[i, itstim:], rout[m1:m2, i])[0] 


                    writer.writerow([pqif, seed, iloop, 0, 0, ci, tau_rec, tau_div, tau_con, tau_chn])

                nombre_subsub_act_pqif = f"simulacion_{num_simulacion}_act_pqif_{pqif}"
                dir_act_pqif = crear_subcarpeta(nombre_subcarpeta_act, nombre_subsub_act_pqif)
                dir_act_pqif_seed = os.path.join(dir_act_pqif, f"simulacion_{num_simulacion}_ej_actividad_semilla_{seed}")
                if not os.path.exists(dir_act_pqif_seed):
                    os.makedirs(dir_act_pqif_seed)
                path = os.path.join(dir_act_pqif_seed, f"simulacion_{num_simulacion}_act_pqif_{pqif}_seed_{seed}_neurona_")

                for iplot in i_graph:
                    plots(iplot, path, amp0, target, iout, pqif, rout)

                





KeyboardInterrupt: 

In [ ]:
if __name__ == "__main__":
    num_simulacion = 12
    nombre_carpeta, nombre_subcarpeta_act, nombre_subcarpeta_pesos, nombre_subcarpeta_corrientes = crear_carpetas(num_simulacion)
    
    target = generate_target()
    
    filename_resultados = f'simulacion_{num_simulacion}_resultados.csv'
    
    csv_file_path = os.path.join(nombre_carpeta, filename_resultados)
    
    column_names = ['pqif', 'seed', 'nloop', 'cc_lif', 'cc_qif', 'cc', 'sigma2', 'tau_rec', 'tau_div', 'tau_con', 'tau_chn']
    crear_archivo_parametros(filename_resultados, num_simulacion, nombre_carpeta)
    pqif_values = [0.5]
    
    with open(csv_file_path, mode='a', newline='') as file:

        writer = csv.writer(file)

        if file.tell() == 0:
            writer.writerow(column_names)

        for pqif in pqif_values:

            nqif = int(N * pqif)

            path_corrientes_seed = os.path.join(nombre_subcarpeta_corrientes, f'simulacion_{num_simulacion}_corrientes_pqif_{pqif}.csv')

            with open(path_corrientes_seed, mode='a', newline='') as file_:

                writer_ = csv.writer(file_)

                if file_.tell() == 0:
                    writer_.writerow(['pqif', 'seed', 'iloop', 'it', 'II_0', 'v_0', 'II_1', 'v_1', 'II_N2+1', 'v_N2+1', 'II_N2+2', 'v_N2+2'])

                
                for seed in range(cant_seed):

                    np.random.seed(seed)

                    x, r, nspike = initialize_neurons(N)

                    rout = np.zeros((1, N))

                    Iext = currents(N, itmax)

                    w = initialize_connectivity_matrix(N, p, gsyn)

                    P, idx = initialize_training(N, w)

                    for iloop in range(nloop):
                        for it in range(itmax):

                            II = np.squeeze(Iext[:, it])
                            v = w.dot(r.T).A1

                            dx, dr = dynamics(x, r, II + v, nqif)

                            xnew = x + dt * dx / 2
                            rnew = r + dt * dr / 2

                            dx, dr = dynamics(xnew, rnew, II + v, nqif)

                            xnew = x + dt * dx
                            rnew = r + dt * dr

                            xnew, rnew, nspike = detect(x, xnew, rnew, nspike, nqif, vt)

                            x, r = xnew.copy(), rnew.copy()
                            rout = np.vstack([rout, r[iout]])

                            if iloop in [nloop_train + 1, nloop - 1] and it % 20 == 0:
                                writer_.writerow([pqif, seed, iloop, it, II[0], v[0], II[1], v[1], II[N2 + 1], v[N2 + 1], II[N2 + 2], v[N2 + 2]])

                        if 0 < iloop <= nloop_train and it > itstim:
                            w, P = learning(it, w, r, P, idx, target)


                        sigma2, tau_rec, tau_div, tau_con, tau_chn = motifs(w, gsyn, N)

                        if iloop in [0, nloop_train + 1]:
                            path_w_seed = os.path.join(nombre_subcarpeta_pesos, f'simulacion_{num_simulacion}_pesos_pqif_{pqif}_matriz_iloop_{iloop}_semilla_{seed}')
                            guardar_matriz_csv(w, path_w_seed)

                        valid_indices = np.where((np.var(target[:, itstim:], axis=1) > 0) & (np.var(rout[1+itstim+iloop*itmax:1+itstim+(iloop+1)*itmax, :], axis=0) > 0))[0]
                        ci = 0
                        for i in valid_indices:
                            m1 = 1 + itstim + iloop * itmax
                            m2 = m1 + itmax - itstim
                            ci += pearsonr(target[i, itstim:], rout[m1:m2, i])[0]
                
                        writer.writerow([pqif, seed, iloop, 0, 0, ci, sigma2, tau_rec, tau_div, tau_con, tau_chn])

                        
                    nombre_subsub_act_pqif = f"simulacion_{num_simulacion}_act_pqif_{pqif}"
                    dir_act_pqif = crear_subcarpeta(nombre_subcarpeta_act, nombre_subsub_act_pqif)
                    dir_act_pqif_seed = os.path.join(dir_act_pqif, f"simulacion_{num_simulacion}_ej_actividad_semilla_{seed}")
                    os.makedirs(dir_act_pqif_seed, exist_ok=True)
                    path = os.path.join(dir_act_pqif_seed, f"simulacion_{num_simulacion}_act_pqif_{pqif}_seed_{seed}_neurona_")
                    for iplot in i_graph:
                        plots(iplot, path, amp0, target, iout, pqif, rout)